In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import os
import torch
import torch.utils.data as data
import torch.utils.data as data_utils
from torchvision import transforms
import torch.nn as nn
import csv
from matplotlib.pylab import plt # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
class ownDatatype(data.Dataset):
    def __init__(self, tensor, transform = None):
        self.tensor = tensor
        self.transform = transform
    def __len__(self):
        return self.tensor[1].size(0)
    def __getitem__(self, index):
        if self.transform is not None:
            X = self.tensor[0][index]
            X = self.transform(X)
        else:
            X = self.tensor[0][index]
        y = self.tensor[1][index]
        return X, y

In [ ]:
def get_dataset_in_torch(dataset,  batch_size, train, tranform = None):
    dataset = dataset[1:]
    label = dataset[:, 0].astype(int)
    dataset = dataset[:, 1:]
    label = torch.from_numpy(label).to(device)
    dataset = torch.from_numpy(dataset).to(device)
    #label = label[:, None]
    dataset = dataset.view(-1, 28, 28)
    dataset = dataset[:, None]
    dataset_as_dataloader = ownDatatype((dataset, label), tranform)
    if train == False:
        loader = data_utils.DataLoader(dataset_as_dataloader, batch_size=batch_size, shuffle=False)
        return loader
    else:
        dataset_size = int(len(dataset_as_dataloader)*0.9)
        validation_size = len(dataset_as_dataloader) - dataset_size 

        train_set, validate_set = data_utils.random_split(dataset_as_dataloader, [dataset_size, validation_size])

        loader = data_utils.DataLoader(train_set, batch_size=batch_size, shuffle=True)
        vali = data_utils.DataLoader(validate_set, batch_size=batch_size, shuffle=True)
        return loader, vali


In [ ]:
class CNNCell(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(CNNCell, self).__init__()
        self.convolutional = nn.Conv2d(in_channels = input_channels, 
                                    kernel_size = 3, out_channels = output_channels,
                                     stride=1, padding=1)
        self.batchn = nn.BatchNorm2d(num_features = output_channels) 
        self.activation = nn.ReLU()
    def forward(self, data):
        output = self.convolutional(data)
        output = self.batchn(output)
        output = self.activation(output)
        return output

In [ ]:
class CNNNetwork(nn.Module):
    def __init__(self):
        super(CNNNetwork, self).__init__()
        self.CNNcell1 = CNNCell(input_channels = 1, output_channels = 30)
        self.CNNcell2 = CNNCell(input_channels = 30, output_channels = 30)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2)
        self.CNNcell3 = CNNCell(input_channels = 30, output_channels = 60)
        self.CNNcell4 = CNNCell(input_channels = 60, output_channels = 60)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2)
        self.CNNcell5 = CNNCell(input_channels = 60, output_channels = 90)
        self.CNNcell6 = CNNCell(input_channels = 90, output_channels = 90)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2)
        
        self.fc1 = nn.Linear(in_features = 90*3*3, out_features = 400)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features = 400, out_features = 300)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(in_features = 300, out_features = 10)

        self.CNNnetwork = nn.Sequential(self.CNNcell1, self.CNNcell2, 
                  self.max_pool1, self.CNNcell3, self.CNNcell4, self.max_pool2, self.CNNcell5, self.CNNcell6, self.max_pool3)
        self.activation = nn.ReLU()
    def forward(self, x):
        x = self.CNNnetwork(x)
        x = x.view(-1, 90*3*3)
        x = self.activation(self.fc1(x))
        x = self.dropout1(x)
        x = self.activation(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


In [ ]:
class Training_class():
    def __init__(self, model, lr, device, wd):
        self.model = model
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr = lr, weight_decay=wd, amsgrad=False)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size = 200, gamma=0.5)
        self.device = device
        self.listLoss_val = []
        self.listLoss_train = []
        self.listAcc_batch_train = []
        self.listAccval = []
        self.listAccTrain = []

    def kaggle_sub(self, validate_loader):
        output = []
        accuracy = []
        epoch_loss = 0
        self.model = self.model.eval()
        for batch_index, (input, label) in enumerate(validate_loader):
            with torch.no_grad():
                label = label.to(self.device)
                input = input.to(self.device)
                label_prediction = self.model(input.float())
                #
                indicies_predicted = torch.argmax(label_prediction, dim=1)
                label = label.cpu().numpy().reshape((len(label), 1)).astype(int)
                indicies_predicted = indicies_predicted.cpu().numpy().reshape((len(label), 1)).astype(int)
                out = np.concatenate((label, indicies_predicted), axis=1).tolist()
                output += out
                #
        return output


    def test(self, batch_size, validate_loader):
        accuracy = []
        epoch_loss = 0
        self.model = self.model.eval()
        for batch_index, (input, label) in enumerate(validate_loader):
            with torch.no_grad():
                label = label.to(self.device)
                input = input.to(self.device)
                label_prediction = self.model(input.float())
                #
                indicies_predicted = torch.argmax(label_prediction, dim=1)
                number_of_wrong = torch.where(label != indicies_predicted, 
                                torch.tensor([1]).to(self.device), torch.tensor([0]).to(self.device))
                accuracy_batch = 1 - torch.sum(number_of_wrong).item() / batch_size #*
                accuracy.append(accuracy_batch)
                #
                loss = self.criterion(label_prediction, label)
                epoch_loss += loss.item()
        self.listLoss_val.append(epoch_loss)
        self.listAccval.append(np.mean(accuracy))
        print("val total loss:{:4} ; val accuracy: {:4}".format(epoch_loss, np.mean(accuracy)))

    def train_funct(self, batch_size, train_loader, validate_loader, test_data, epochs):
        print("hm")
        for epoch in range(epochs):
            self.model = self.model.train()
            epoch_loss = 0
            epoch_accuracy = []
            for batch_index, (input, label) in enumerate(train_loader):
                self.optimizer.zero_grad()
                input = input.to(self.device)
                label = label.to(self.device)
                label_prediction = model(input.float())
                #
                indicies_predicted = torch.argmax(label_prediction, dim=1)
                number_of_wrong = torch.where(label != indicies_predicted, 
                                torch.tensor([1]).to(self.device), torch.tensor([0]).to(self.device))
                accuracy_batch = 1 - torch.sum(number_of_wrong).item() / batch_size
                epoch_accuracy.append(accuracy_batch)
                #
                loss = self.criterion(label_prediction, label)
                epoch_loss += loss.item()
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()
                self.listAcc_batch_train.append(accuracy_batch)
            print("Epoch now:{} ; training total loss:{:4} ; training accuracy: {:4}".format(epoch, epoch_loss, np.mean(epoch_accuracy)))
            self.test(batch_size, validate_loader)
            self.listLoss_train.append(epoch_loss)
            self.listAccTrain.append(np.mean(epoch_accuracy))
        print("This is test data")
        self.test(batch_size, test_data)
        return model

In [ ]:
transform = transforms.Compose([transforms.Normalize((0.5,),(0.5,))]) # can add more
transform = None
batch_size = 100

path = "../input/Kannada-MNIST/"

train_data = np.genfromtxt(path +'train.csv', delimiter=',')
train_data, validation_data = get_dataset_in_torch(train_data, batch_size, True, transform)

test_data = np.genfromtxt(path +'Dig-MNIST.csv', delimiter=',')
test_data = get_dataset_in_torch(test_data, batch_size, False, transform)

kaggle_data = np.genfromtxt(path +'test.csv', delimiter=',')
kaggle_data = get_dataset_in_torch(kaggle_data, batch_size, False, transform)

In [ ]:
model = CNNNetwork()
model.to(device)

epochs = 5
lr = 3e-4
wd = 1e-2

train_class = Training_class(model, lr, device, wd)

model = train_class.train_funct(batch_size, train_data, validation_data, test_data, epochs)
output = train_class.kaggle_sub(kaggle_data)

In [ ]:
sub=pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
labels = []
for i in range(len(output)):
    labels.append(output[i][1])

sub['label']=labels
sub.to_csv('submission.csv',index=False)

sub.head()

